# Radiative Forcing of different model simulations

https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2016GL071930

paths of namelist files:

CTRL: `/projects/0/prace_imau/prace_2013081679/cesm1_0_4/spinup_pd_maxcores_f05_t12/run/atm_in`

LPD: `/projects/0/acc/cesm/cesm1_1_2/spinup_B_2000_cam5_f09_g16/run/atm_in`

"present day"/"year 2000" CO<sub>2</sub> value: 367 ppm

In [ ]:
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import xarray as xr
import cmocean
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
from paths import path_results

In [ ]:
def radiative_forcing(C, N, M):
    """
    C .. CO2 conc. [ppm]
    M .. CH4 conc. [ppb]
    N .. N2O conc. [ppb]
    """
    a1 = -2.4e-7
    b1 =  7.2e-4
    c1 = -2.1e-4
    a2 = -8.0e-6
    b2 =  4.2e-6
    c2 = -4.9e-6
    a3 = -1.3e-6
    b3 = -8.2e-6
    
    C0 = 278.  # [ppm]
    M0 = 722.  # [ppb]
    N0 = 270.  # [ppb]
    
    Cbar = (C+C0)/2  # [ppm]
    Mbar = (M+M0)/2  # [ppb]
    Nbar = (N+N0)/2  # [ppb]
    
    CO2_forcing = (a1*(C-C0)**2 + b1*abs(C-C0) + c1*Nbar + 5.36) * np.log(C/C0)
    N2O_forcing = (a2*Cbar + b2*Nbar + c2*Mbar + 0.177) * (np.sqrt(N) - np.sqrt(N0))
    CH4_forcing = (a3*Mbar + b3*Nbar + 0.043) * (np.sqrt(M) - np.sqrt(M0))

    return (CO2_forcing, N2O_forcing, CH4_forcing)


def forcings(ds):
    ds['CO2_forc'] = ds['CO2'].copy()
    ds['CH4_forc'] = ds['CO2'].copy()
    ds['N2O_forc'] = ds['CO2'].copy()
    for t in range(len(ds.time)):
        (CO2, N2O, CH4) = radiative_forcing(C=ds.CO2[t], N=ds.N2O[t], M=ds.CH4[t])
        ds['CO2_forc'][t] = CO2
        ds['CH4_forc'][t] = CH4
        ds['N2O_forc'][t] = N2O
    return ds

In [ ]:
def onepercent(CO20):
    A = np.array([CO20]*140)
    for t in np.arange(1,140):
        A[t] = A[t-1]*1.01
    return A

A = onepercent(278.)
B = onepercent(369.)

In [ ]:
plt.plot(A)
plt.plot(B)


## RCP8.5 GHG forcings

In [ ]:
rcp85 = '/projects/0/prace_imau/prace_2013081679/cesm1_0_4/inputdata/atm/cam/ggas/make_new_ghg/ghg_rcp85_1765-2500_c100203.nc'

In [ ]:
ds = xr.open_dataset(rcp85, decode_times=False)

In [ ]:
ds = forcings(ds)

In [ ]:
ds

In [ ]:
f, ax = plt.subplots(2, 3, figsize=(12,5), sharex='col')

# plt.plot(ds.time/365+1765, ds.CH4)
for i in range(2):
    for j in range(3):
        ax[1,j].set_xlabel('year', fontsize=14)
        ax[1,j].set_xlim((1850,2120))
        ax[i,j].tick_params(labelsize=14)
        ax[i,j].axvline(2000, c='k', lw=.5, ls=':')
        ax[i,j].axvline(2100, c='k', lw=.5, ls=':')


ax[0,0].set_ylabel('CO2 [ppmv]', fontsize=14)
ax[0,0].plot(ds.time/365+1765, ds.CO2)
ax[0,0].axhline(ds.CO2.sel({'time':(2000-1765)*365}, method='nearest'), ls=':')
ax[0,0].axhline(ds.CO2.sel({'time':(1765-1765)*365}, method='nearest'), ls='--')
ax[0,0].plot(np.arange(2000,2000+140), A, c='r')
ax[0,0].plot(np.arange(2000,2000+140), B, c='r', ls='--')

ax[0,0].scatter(2018,410)
ax[0,1].set_ylabel('CH4 [ppbv]', fontsize=14)
ax[0,1].plot(ds.time/365+1765, ds.CH4)
ax[0,1].axhline(ds.CH4.sel({'time':(2000-1765)*365}, method='nearest'), ls=':')
ax[0,1].axhline(ds.CH4.sel({'time':(1765-1765)*365}, method='nearest'), ls='--')
ax[0,1].scatter(2018,1850)
ax[0,2].set_ylabel('N20 [ppbv]', fontsize=14)
ax[0,2].plot(ds.time/365+1765, ds.N2O)
ax[0,2].axhline(ds.N2O.sel({'time':(2000-1765)*365}, method='nearest'), ls=':')
ax[0,2].axhline(ds.N2O.sel({'time':(1765-1765)*365}, method='nearest'), ls='--')
ax[0,2].scatter(2018,333)


ax[1,0].set_ylabel('CO2 forcing [W/m^2]', fontsize=14)
ax[1,0].plot(ds.time/365+1765, ds.CO2_forc)
ax[1,0].axhline(ds.CO2_forc.sel({'time':(2000-1765)*365}, method='nearest'), ls=':')
ax[1,0].axhline(ds.CO2_forc.sel({'time':(1765-1765)*365}, method='nearest'), ls='--')
ax[1,1].set_ylabel('CH4 forcing [W/m^2]', fontsize=14)
ax[1,1].plot(ds.time/365+1765, ds.CH4_forc)
ax[1,1].axhline(ds.CH4_forc.sel({'time':(2000-1765)*365}, method='nearest'), ls=':')
ax[1,1].axhline(ds.CH4_forc.sel({'time':(1765-1765)*365}, method='nearest'), ls='--')
ax[1,2].set_ylabel('N20 forcing [W/m^2]', fontsize=14)
ax[1,2].plot(ds.time/365+1765, ds.N2O_forc)
ax[1,2].axhline(ds.N2O_forc.sel({'time':(2000-1765)*365}, method='nearest'), ls=':')
ax[1,2].axhline(ds.N2O_forc.sel({'time':(1765-1765)*365}, method='nearest'), ls='--')

ax[0,0].set_ylim((250,1000))
ax[1,0].set_ylim((-.5,9))
f.align_ylabels()
f.tight_layout()
f.savefig(f'{path_results}/FORCING/RCP85_forcing')